In [5]:
import os
import json
from autogen import ConversableAgent
from bs4 import BeautifulSoup

# Define your model configuration
api_key = os.environ.get("GOOGLE_API_KEY")
config_list_gemini = [{"model": "gemini-1.5-flash-latest","api_key": api_key,"api_type": "google"}]

llm_config = {"config_list": config_list_gemini,"seed": 42}
google_html_jobs_dir = '/Users/vimal.chaudhari/Downloads/vimalKeshu/job-search/backend/crawler/google/jobs/'
google_json_jobs_dir = '/Users/vimal.chaudhari/Downloads/vimalKeshu/job-search/backend/crawler/google/jobs/json'
job_url = "https://www.google.com/about/careers/applications/jobs/results/{}"

html_parser_prompt = '''
You are html parser.you here to retrieve the text information from the given html page if present.
Provide the answer in below python parsable json string:
{"title": job title as string else empty string, "level": job level as string else empty string,"location": job location as string else empty string,"description": about job description as string else empty string,"key_qualifications": job Minimum qualifications as string or empty string, "preferred_qualifications": job preferred qualifications as string else empty string, "responsibilities": job Responsibilities as string else empty string}
Do not ask for other information. If you don't get information, return answer with empty json.
'''

def load_visited_files(dir=google_json_jobs_dir)-> set:
    visited:set = set()
    for file in os.listdir(dir):
        jid = file.replace('.json','')
        visited.add(jid)
    return visited

html_parser_agent = ConversableAgent(
    name="html_parser",
    system_message=html_parser_prompt,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1
)

In [6]:
cnt = 0
visited:set = load_visited_files()

for html_file_name in os.listdir(google_html_jobs_dir):
    google_job_html_file_path = os.path.join(google_html_jobs_dir, html_file_name)
    page_str=''
    job_id = html_file_name.replace('.html','')

    if job_id in visited:
        print('visited: '+job_id)
        continue

    with open(google_job_html_file_path, 'r') as file:
        page_str = file.read()
    
    soup = BeautifulSoup(page_str, "html.parser")
    results = soup.find_all("div", attrs={'data-id': job_id})
    #print(str(results[0]))
    if not results:
        continue
    html_str=str(results[0])

    reply = html_parser_agent.generate_reply(
        messages=[{"content": html_str, "role": "user"}]
    )

    if reply:
        message = reply['content']
        message = message.replace("```json",'')
        message = message.replace("```",'')
        job_json = json.loads(message)
        job_json['company'] = 'google'
        job_json['url'] = job_url.format(job_id)
        job_json_str = json.dumps(job_json)
        print(job_json_str)
        json_path = os.path.join(google_json_jobs_dir, job_id + '.json')
        with open(json_path, 'w') as file:
            file.write(job_json_str)
        print('written: ' + json_path)
    # if cnt > 2:
    #     break 
    # cnt+=1

visited: 103862064232964806
visited: 134909408730587846
visited: 122209165941580486


I0000 00:00:1722183405.906872 7157978 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


RuntimeError: Google GenAI exception occurred while calling Gemini API: index: 0
finish_reason: RECITATION
